In [23]:
using CSV
using DataFrames
using LinearAlgebra


network_df = CSV.read("../SiouxFalls/SiouxFalls_net.txt", DataFrame; delim='\t')
network_df = select(network_df, Not(:Column1))
node_df = CSV.read("../SiouxFalls/SiouxFalls_node.txt", DataFrame; delim='\t')
node_df = select(node_df, Not(";"))
od_df = CSV.read("../SiouxFalls/SiouxFalls_od.csv", DataFrame)

od = [(row.O, row.D) for row in eachrow(od_df)] 
road_link = [(row.init_node, row.term_node, row.length) for row in eachrow(network_df)]
A = [(row.init_node, row.term_node) for row in eachrow(network_df)] #arcs


76-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (2, 1)
 (2, 6)
 (3, 1)
 (3, 4)
 (3, 12)
 (4, 3)
 (4, 5)
 (4, 11)
 (5, 4)
 (5, 6)
 (5, 9)
 ⋮
 (21, 22)
 (21, 24)
 (22, 15)
 (22, 20)
 (22, 21)
 (22, 23)
 (23, 14)
 (23, 22)
 (23, 24)
 (24, 13)
 (24, 21)
 (24, 23)

In [24]:
alpha = 0.15
beta = 4
t0_a = Dict((row.init_node, row.term_node) => row.free_flow_time for row in eachrow(network_df))
c_a = Dict((row.init_node, row.term_node) => row.capacity for row in eachrow(network_df))


Dict{Tuple{Int64, Int64}, Float64} with 76 entries:
  (18, 16) => 19679.9
  (14, 23) => 4924.79
  (23, 24) => 5078.51
  (11, 10) => 10000.0
  (22, 15) => 9599.18
  (7, 18)  => 23403.5
  (7, 8)   => 7841.81
  (3, 12)  => 23403.5
  (8, 9)   => 5050.19
  (14, 15) => 5127.53
  (19, 20) => 5002.61
  (2, 1)   => 25900.2
  (10, 17) => 4993.51
  (6, 5)   => 4948.0
  (20, 22) => 5075.7
  (16, 17) => 5229.91
  (17, 19) => 4823.95
  (16, 18) => 19679.9
  (16, 8)  => 5045.82
  (20, 18) => 23403.5
  (16, 10) => 4854.92
  (21, 20) => 5059.91
  (19, 17) => 4823.95
  (4, 5)   => 17782.8
  (4, 11)  => 4908.83
  ⋮        => ⋮

In [33]:
using LightGraphs, GraphsFlows

function generate_route_sets_link_elimination(graph, source, target, num_routes)
    route_sets = []
    for i in 1:num_routes
        path = dijkstra_shortest_paths(graph, source, target).path
        if !(path in route_sets)
            push!(route_sets, path)
        end
        
        if length(path) > 2
            edge_to_remove = (path[2], path[3])
            original_weight = graph.weights[edge_to_remove]
            graph.weights[edge_to_remove] = Inf
        else
            break
        end
    end
    
    # Reset graph weights for future usage
    # Note: Implement resetting of weights as per your graph implementation

    return route_sets
end

function generate_route_sets_link_penalty(graph, source, target, num_routes, penalty_factor=1.5)
    route_sets = []
    for i in 1:num_routes
        path = dijkstra_shortest_paths(graph, source, target).path
        if !(path in route_sets)
            push!(route_sets, path)
        end
        
        for j in 1:length(path) - 1
            edge = (path[j], path[j+1])
            graph.weights[edge] *= penalty_factor
        end
    end
    
    # Reset graph weights for future usage
    # Note: Implement resetting of weights as per your graph implementation

    return route_sets
end


generate_route_sets_link_penalty (generic function with 2 methods)

In [47]:
# https://juliagraphs.org/Graphs.jl/dev/

# we still need to convert following python code to Julia

# n_routes = 3
for od_pair_index in range(len(od_df)):
    i,j = od_df['O'].iloc[od_pair_index], od_df['D'].iloc[od_pair_index]
    G1 = nx.DiGraph()
    G1.add_weighted_edges_from(road_link)
    route_sets1 = generate_route_sets_link_penalty(G1, i, j, n_routes) 
    OD_route[(i,j)] = route_sets1 

LoadError: UndefVarError: `SimpleDiGraph` not defined

In [26]:
using JuMP
using MosekTools

# Model initialization
model = Model(Mosek.Optimizer)

# Variables
@variable(model, x_a[a in A] >= 0)
@variable(model, mu_a[a in A] >= 0)
@variable(model, theta_a[a in A] >= 0)
@variable(model, u_a[a in A] >= 0)
# TODO: define v_p

# Objective function
@objective(model, Min, sum(t0_a[a] * x_a[a] + (t0_a[a] * alpha / (beta+1)/ c_a[a]) * mu_a[a] for a in A))

# Constraint: Sum of v_p over paths in P_{od} equals 1 for each origin-destination pair
@constraint(model, [o in O, d in D], sum(v_p[p] for p in P_od[(o, d)]) == 1)

# Constraint: x_a equals the sum of d_{od} * v_p over all o, d, and paths that include arc a
@constraint(model, [a in A], x_a[a] == sum(d_od[(o, d)] * v_p[p] for o in O, d in D, p in P_od[(o, d)] if a in p))

# Non-negativity constraints for v_p and x_a
@constraint(model, [o in O, d in D, p in P_od[(o, d)]], v_p[p] >= 0)
@constraint(model, [a in A], x_a[a] >= 0)


# Constraints
for a in A
    @constraint(model, [2 * x_a[a], theta_a[a] - 1, theta_a[a] + 1] in SecondOrderCone())
    @constraint(model, [2 * theta_a[a], u_a[a] - x_a[a], u_a[a] + x_a[a]] in SecondOrderCone())
    @constraint(model, [2 * u_a[a], mu_a[a] - x_a[a], mu_a[a] + x_a[a]] in SecondOrderCone())
end

# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)

if status == MOI.OPTIMAL
    # Extracting the solution
    x_a_solution = value.(x_a)
    mu_a_solution = value.(mu_a)
    theta_a_solution = value.(theta_a)
    u_a_solution = value.(u_a)
    println("Optimal solution found")
    # You can print or use the solutions as needed
else
    println("Problem status: $status")
end


Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 0               
  Affine conic cons.     : 228 (684 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 304             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes      